In [ ]:
import json
import websocket
import threading
import time
import requests

# 모의투자용 APP_KEY와 APP_SECRET 입력
APP_KEY = "여기_입력"
APP_SECRET = "여기_입력"

# approval_key 발급 함수 (모의투자 전용 URL)
def get_approval_key(appkey, appsecret):
    url = "https://openapivts.koreainvestment.com:29443/oauth2/tokenP"
    headers = {"content-type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "appkey": appkey,
        "appsecret": appsecret
    }
    res = requests.post(url, headers=headers, data=data)
    res_json = res.json()
    print("응답:", res_json)
    if "approval_key" not in res_json:
        raise Exception(f"approval_key 없음. 오류 코드: {res_json.get('error_code')}, 설명: {res_json.get('error_description')}")
    return res_json["approval_key"]

# 실시간 시세 수신 핸들러 함수
def on_message(ws, message):
    data = json.loads(message)
    if 'body' in data:
        body = data['body']
        symbol = body.get('symbol', '')
        price = float(body.get('stck_prpr', body.get('fut_prpr', 0)))
        prices[symbol] = price
        print(f"{symbol}: {price}")

        try:
            call = prices[SYMBOL_CALL]
            put = prices[SYMBOL_PUT]
            fut = prices[SYMBOL_FUTURE]
            k = STRIKE_PRICE
            diff = (call - put) - (fut - k)
            print(f"콜: {call}, 풋: {put}, 선물: {fut} → 차익 = {diff:.2f}")
            if abs(diff) > 0.5:
                print("차익거래 기회 탐지!")
        except KeyError:
            pass

# 기타 WebSocket 핸들러
def on_error(ws, error):
    print("에러:", error)

def on_close(ws, close_status_code, close_msg):
    print("연결 종료")

def on_open(ws):
    print("WebSocket 연결 성공")
    for symbol in [SYMBOL_CALL, SYMBOL_PUT, SYMBOL_FUTURE]:
        body = {
            "header": {
                "approval_key": approval_key,
                "custtype": "P",
                "tr_type": "1",
                "content-type": "utf-8"
            },
            "body": {
                "input": {
                    "tr_id": "FHKST01010100" if symbol.startswith("1") else "OPTKST01010100",
                    "tr_key": symbol
                }
            }
        }
        ws.send(json.dumps(body))
        print(f"{symbol} 구독 요청 완료")

# 종목 코드 (7월 만기 선물 + 콜/풋옵션)
SYMBOL_FUTURE = "101W07"
SYMBOL_CALL = "201W07340"
SYMBOL_PUT = "301W07340"
STRIKE_PRICE = 340.0

# 실시간 가격 저장용 딕셔너리
prices = {}

# approval_key 발급
approval_key = get_approval_key(APP_KEY, APP_SECRET)
print("approval_key 발급 완료:", approval_key)

# WebSocket 연결
socket_url = "wss://openapivts.koreainvestment.com:7443/websocket"
ws = websocket.WebSocketApp(socket_url,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

# WebSocket 실행 스레드로 시작
thread = threading.Thread(target=ws.run_forever)
thread.start()